# Chat with prompty


**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using prompty and visualize the trace of your application.
- Understand how to handle chat conversation using prompty
- batch run prompty against multi lines of data.


## 0. Install dependent packages

In [1]:
%%capture --no-stderr
%pip install promptflow-devkit

In [9]:
%pip install promptflow promptflow-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for google-search-results: filename=google_search_results-2.4.1-py3-none-any.whl size=25775 sha256=005a5ed260b26a486d15f24ebfb7af025174c0742f2bab2535dc5466ad098f48
  Stored in directory: /home/azureuser/.cache/pip/wheels/4d/3c/ae/8e980479459000ba63d0173190f01a56a6cd8c812c338efeb5
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.


## 1. Prompty

Prompty is a file with .prompty extension for developing prompt template. 
The prompty asset is a markdown file with a modified front matter. 
The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [9]:
with open("prompty/chat.prompty") as fin:
    print(fin.read())

---
name: Chat Prompt
description: A basic prompt that uses the chat API to answer questions with chat_history
model:
    api: chat
    configuration:
        type: azure_openai
        connection: my_azure_open_ai_connection
        azure_deployment: gpt-4-0125-Preview
    parameters:
        max_tokens: 256
        temperature: 0.2

inputs:
    question:
        type: string
    chat_history:
        type: list
        default: []
sample:
    question: What is the meaning of life?
    chat_history: []

---
system:
You are an AI assistant who helps people find information.
As the assistant, you answer questions briefly, succinctly, 
and in a personable manner using markdown and even add some personal flair with appropriate emojis.

{% for item in chat_history %}
{{item.role}}:
{{item.content}}
{% endfor %}

user:
{{question}}


### Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

Above prompty uses connection `open_ai_connection` inside, we need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

Prepare your Azure Open AI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

In [12]:
%pip install keyrings.alt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from promptflow.client import PFClient
from promptflow.connections import AzureOpenAIConnection, OpenAIConnection

from promptflow.entities import AzureOpenAIConnection
client = PFClient()
# Initialize an AzureOpenAIConnection object
connection = AzureOpenAIConnection(
    name="my_azure_open_ai_connection",
    api_key="8b96d7ba6a31403089100421919c7962",
    api_base="https://azuremlopenai.openai.azure.com/",
)
# Create the connection, note that api_key will be scrubbed in the returned result
result = client.connections.create_or_update(connection)
print(result)

print(connection)

auth_mode: key
name: my_azure_open_ai_connection
module: promptflow.connections
created_date: '2024-07-04T01:25:55.056628'
last_modified_date: '2024-07-04T01:26:05.599050'
type: azure_open_ai
api_key: '******'
api_base: https://azuremlopenai.openai.azure.com/
api_type: azure
api_version: '2024-02-01'

auth_mode: key
name: my_azure_open_ai_connection
module: promptflow.connections
type: azure_open_ai
api_key: '******'
api_base: https://azuremlopenai.openai.azure.com/
api_type: azure
api_version: '2024-02-01'



In [6]:
conn_name = "my_azure_open_ai_connection"
conn = client.connections.get(name=conn_name)
print("using this connection :",conn_name)

using this connection : my_azure_open_ai_connection


### Execute prompty as function

In [10]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load("prompty/chat.prompty")
# execute the flow as function
question = "What is the capital of France?"
result = f(question=question)
result

"The capital of France is Paris! 🇫🇷✨ It's not only the country's capital but also a global hub for art, fashion, gastronomy, and culture. A truly iconic city!"

You can override connection with `AzureOpenAIModelConfiguration` and `OpenAIModelConfiguration`.

In [12]:
from promptflow.core import AzureOpenAIModelConfiguration, OpenAIModelConfiguration


# override configuration with created connection in AzureOpenAIModelConfiguration
configuration = AzureOpenAIModelConfiguration(
    connection="my_azure_open_ai_connection", azure_deployment="gpt-4o"
)

# override openai connection with OpenAIModelConfiguration
# configuration = OpenAIModelConfiguration(
#     connection=connection,
#     model="gpt-3.5-turbo"
# )

override_model = {
    "configuration": configuration,
}

# load prompty as a flow
f = Prompty.load("prompty/chat.prompty", model=override_model)
# execute the flow as function
question = "What is the capital of France?"
result = f(question=question)
result

'The capital of France is Paris! 🇫🇷✨'

### Visualize trace by using start_trace

In [13]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Prompt flow service has started...


Re-run below cell will collect a trace in trace UI.

In [14]:
# rerun the function, which will be recorded in the trace
result = f(question=question)
result

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=AzureOpenAI_Advanced&uiTraceId=0x1a838b0c75a1507d63de6faf4bb52c8b


'The capital of France is Paris! 🇫🇷✨'

In [23]:
question = "Summarize our conversation"
result = f(question=question)
result

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=AzureOpenAI_Advanced&uiTraceId=0x14a49a99ac5034d99bb133fa4bed581e


"Sure thing! 😊\n\nYou asked me to summarize our conversation. So far, we've been discussing how I can help you find information and answer your questions in a brief, succinct, and friendly manner using markdown. 📚✨\n\nFeel free to ask me anything else!"

### Eval the result 

In this example, we will use a prompt that determines whether a chat conversation contains an apology from the assistant.

In [19]:
eval_prompty = "prompty/apology.prompty"

with open(eval_prompty) as fin:
    print(fin.read())

---
name: Apology Prompt
description: A prompt that determines whether a chat conversation contains an apology from the assistant
model:
  api: chat
  configuration:
    type: azure_openai
    connection: my_azure_open_ai_connection
    azure_deployment: gpt-4o
  parameters:
    temperature: 0.2
    response_format: { "type": "json_object" }
inputs: 
  question:
    type: string
  answer:
    type: string
  messages:
    type: list
outputs:
  apology:
    type: string
sample: ${file:sample.json}
---

system:
You are an AI tool that determines if, in a chat conversation, the assistant apologized, like say sorry.
Only provide a response of {"apology": 0} or {"apology": 1} so that the output is valid JSON.
Give a apology of 1 if apologized in the chat conversation.

Here are some examples of chat conversations and the correct response:

**Example 1**
user: Where can I get my car fixed?
assistant: I'm sorry, I don't know that. Would you like me to look it up for you?
result:
{"apology": 1}

Note: the eval flow returns a `json_object`.

In [24]:
# load prompty as a flow
eval_flow = Prompty.load(eval_prompty)
# execute the flow as function
result = eval_flow(question=question, answer=result, messages=[])
result

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=AzureOpenAI_Advanced&uiTraceId=0xcf3e06f98aa514381d8dc3d2d6f877ce


{'apology': 0}